### Data munging (General)

#### Importing libraries and source files

In [49]:

import xlwings as xw
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [50]:
# --  Choose which file to import from below
extended_df_name = 'extended_fmeca.xlsx' # The whole fmeca - which has all the newly calculated columns
#req_calc_name = 'fmeca_req_calc.xlsx' # A slice of the fmeca with selected columns that require calculation. The order of columns is important here.
#correct_abts_name = 'correct_abts.xlsx' # A slice of the fmeca which requires abt corrections.
#correct_dim_name = 'correct_dim.xlsx' # A slice of fmeca which requires correct diminution levels

# -- File path to import from
file_location = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working/ProcessedData'

In [51]:
df = pd.read_excel(os.path.join(file_location,extended_df_name),sheet_name='Sheet1')

In [52]:
df.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Asbuilt,Min,Pit %,Location,Detail,Repair,Inspection interval,Column1,Assessment notes,Adj Final action,Coating condition,Flag_2,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated,repair_formulate,inspInt_formulate,allowables,allowableCheck,calc_y_n_formulate,Anomlay ID,stiff_anom_combo
0,1,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD-CR-192,AW,49-50,LL15,LBHD,4200,100,FR.49,500.0,LL15,0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,NaN,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7,8.0,280.0,NaN,0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0,0.477692,Local,Med,Med,Y,Generalised corrosion,13.0,3.7,0.715385,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,-,NaN,Local Generalised corrosion on the longitudina...,NaN,Poor,R,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,-,0.2,correct dim level,Y,4SWBT-LBHD-CR-192,LL15_4SWBT-LBHD-CR-192


### Get the pass fail data from calc sheet

In [53]:
# --  Choose which file to import from below
calc_file = 'FSC-JAD-01-TEC-62-A1 MV LBHD Calcs AN.xlsm' # The whole fmeca - which has all the newly calculated columns

# -- File path to import from
file_location = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets'
file_path = os.path.join(file_location,calc_file)
# -- sheet name
calc_sheet_name = '4 Stress and buckling calcs'
calc_sheet_range = 'A5:IN92'

In [54]:

book = xw.Book(file_path).sheets[calc_sheet_name].range(calc_sheet_range)
df2 = book.options(pd.DataFrame, index=False, header=True).value

In [55]:
#df2 = df2.transpose()

In [82]:
#df2.iloc[[1,86]].transpose().reset_index(drop=True)
def operation1(df):
    calc = df.iloc[[86]].iloc[:,4:].transpose().reset_index(drop=False)
    calc.rename(columns={'index':'stiff_anom_combo',86:'pass_fail'},inplace=True)
    return calc

calc = operation1(df2)

In [84]:
pass_fail_dict = dict(zip(calc['stiff_anom_combo'],calc['pass_fail']))

#### Apply the pass fail results as column into the fmeca sheet

In [90]:
def PassFail(row):
    combo_name = row['stiff_anom_combo']
    try:
        return pass_fail_dict[combo_name]
    except:
        return 'No calc required'

In [91]:
df['pass_fail'] = df.apply(PassFail,axis=1)

#### Notes function